<a href="https://colab.research.google.com/github/anirbansen3027/SecureAndPrivateAI/blob/master/4_PATEforMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Importing libraries
import torch
#library for neural network
from torch import nn
#For weight optimisation
from torch import optim
import torch.nn.functional as F
#For datasets and transformation of dataset
from torchvision import datasets,transforms
#Reading the data

#Transforming the data
transform = transforms.Compose([transforms.ToTensor(),
transforms.Normalize((0.5,), (0.5,))
])

#Reading the dataset to tensor
trainset = datasets.MNIST('~/.pytorch/MNIST_data/',download=True,train=True,transform=transform)
#loading in batch of 64
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64,shuffle=True)
#Creating the model
model = nn.Sequential(nn.Linear(784,128), nn.ReLU(),nn.Linear(128,64),nn.ReLU(),nn.Linear(64,10),nn.LogSoftmax(dim=1))
#Cretaing the Negative Log Loss
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr =0.003)

#Running the model for 5 epochs
epochs = 5
for e in range(epochs):
    running_loss = 0
    for images,labels in trainloader:
        #Reshaping the images
        images = images.view(images.shape[0],-1)
        #Setting the gradients to zero else they add up
        optimizer.zero_grad() 
        #Forward Propogation
        output = model.forward(images)
        #Calculate the loss
        loss = criterion(output,labels)
        #Backward propogate
        loss.backward()
        #Update the weights
        optimizer.step()
        #Calculate total loss
        running_loss += loss.item()
    else:
        print("Training Loss:",(running_loss/len(trainloader)))   
        
#Reading the dataset to tensor
testset = datasets.MNIST('~/.pytorch/MNIST_data/',download=True,train=False,transform=transform)
#loading in batch of 64
testloader = torch.utils.data.DataLoader(trainset, batch_size = 64,shuffle=True)
preds = []
indices = []
for images,labels in testloader:
        #Reshaping the images
        images = images.view(images.shape[0],-1)
        ps = torch.exp(model(images))
        top_p,top_class = ps.topk(1,dim=1)
        preds.extend(top_class)
        indices.extend(labels)

Training Loss: 1.8817425864591781
Training Loss: 0.8119523299655427
Training Loss: 0.5116383800310875
Training Loss: 0.4227397760698028
Training Loss: 0.38149392215618444


In [0]:
import numpy as np
preds = np.asarray(preds)
indices = np.asarray(indices)

In [0]:
teacher_preds = np.reshape(preds, (-1, preds.shape[0]))
teacher_preds.shape

(1, 60000)

In [0]:
!pip install syft
from syft.frameworks.torch.differential_privacy import pate

In [0]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds = teacher_preds, indices = indices,noise_eps = 0.05, delta = 1e-5,moments =20)

In [0]:
data_dep_eps, data_ind_eps

(2411.5129254633675, 2411.5129254649705)